**Variables Instrumentales - Mínimos Cuadrados en 2 Estapas**

In [1]:
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.stats.api as sms

In [2]:
data = pd.read_csv(r"C:\Users\57317\Desktop\Universidad Nacional\Programación\Python\Curso Econometría básica con Python\Bases de datos\MROZ.csv")
data.head()

,inlf,hours,kidslt6,kidsge6,age,educ,wage,repwage,hushrs,husage,...,faminc,mtr,motheduc,fatheduc,unem,city,exper,nwifeinc,lwage,expersq
0,1,1610,1,0,32,12,3.3540,2.65,2708,34,...,16310,0.7215,12,7,5.0,0,14,10.910060,1.210154,196
1,1,1656,0,2,30,12,1.3889,2.65,2310,30,...,21800,0.6615,7,7,11.0,1,5,19.499981,0.328512,25
2,1,1980,1,3,35,12,4.5455,4.04,3072,40,...,21040,0.6915,12,7,5.0,0,15,12.039910,1.514138,225
3,1,456,0,3,34,12,1.0965,3.25,1920,53,...,7300,0.7815,7,7,5.0,0,6,6.799996,0.092123,36
4,1,1568,1,2,31,14,4.5918,3.60,2000,32,...,27300,0.6215,12,14,9.5,1,7,20.100060,1.524272,49


In [3]:
data.isna().sum()

inlf          0
hours         0
kidslt6       0
kidsge6       0
age           0
educ          0
wage        325
repwage       0
hushrs        0
husage        0
huseduc       0
huswage       0
faminc        0
mtr           0
motheduc      0
fatheduc      0
unem          0
city          0
exper         0
nwifeinc      0
lwage       325
expersq       0
dtype: int64

In [4]:
data["wage"].notna()

0       True
1       True
2       True
3       True
4       True
       ...  
748    False
749    False
750    False
751    False
752    False
Name: wage, Length: 753, dtype: bool

In [5]:
data = data[data["wage"].notna()]
data.head()

,inlf,hours,kidslt6,kidsge6,age,educ,wage,repwage,hushrs,husage,...,faminc,mtr,motheduc,fatheduc,unem,city,exper,nwifeinc,lwage,expersq
0,1,1610,1,0,32,12,3.3540,2.65,2708,34,...,16310,0.7215,12,7,5.0,0,14,10.910060,1.210154,196
1,1,1656,0,2,30,12,1.3889,2.65,2310,30,...,21800,0.6615,7,7,11.0,1,5,19.499981,0.328512,25
2,1,1980,1,3,35,12,4.5455,4.04,3072,40,...,21040,0.6915,12,7,5.0,0,15,12.039910,1.514138,225
3,1,456,0,3,34,12,1.0965,3.25,1920,53,...,7300,0.7815,7,7,5.0,0,6,6.799996,0.092123,36
4,1,1568,1,2,31,14,4.5918,3.60,2000,32,...,27300,0.6215,12,14,9.5,1,7,20.100060,1.524272,49


**Proceso Manual**

In [6]:
Y1 = data["lwage"] # Variable dependiente 
Y2 = data["educ"] # Variable endógena 
X = data[["exper","expersq"]] # Variables exógenas
Z = data[["motheduc","fatheduc"]] # Instrumentos

**Primera Etapa**

In [7]:
frames = [X , Z]
regresoras_PE = pd.concat(frames, axis = 1)
regresoras_PE

,exper,expersq,motheduc,fatheduc
0,14,196,12,7
1,5,25,7,7
2,15,225,12,7
3,6,36,7,7
4,7,49,12,14
...,...,...,...,...
423,2,4,7,7
424,21,441,7,7
425,22,484,7,7
426,14,196,12,16


In [8]:
primera_etapa = sm.OLS(Y2, sm.add_constant(regresoras_PE))
resultados_pe = primera_etapa.fit()
print(resultados_pe.summary())

                            OLS Regression Results                            
Dep. Variable:                   educ   R-squared:                       0.211
Model:                            OLS   Adj. R-squared:                  0.204
Method:                 Least Squares   F-statistic:                     28.36
Date:                Fri, 24 Jun 2022   Prob (F-statistic):           6.87e-21
Time:                        10:08:41   Log-Likelihood:                -909.72
No. Observations:                 428   AIC:                             1829.
Df Residuals:                     423   BIC:                             1850.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          9.1026      0.427     21.340      0.0

In [9]:
hipotesis = "(fatheduc = 0), (motheduc = 0)"
prueba_F = resultados_pe.f_test(hipotesis)
print(prueba_F)

<F test: F=array([[55.40030043]]), p=4.268908724630835e-22, df_denom=423, df_num=2>


In [10]:
estadistico = prueba_F.statistic
estadistico

array([[55.40030043]])

In [11]:
pvalue = prueba_F.pvalue
pvalue

array(4.26890872e-22)

In [12]:
nombres = ["Estadístico","P valor"]
pd.Series([estadistico, pvalue], index = nombres)

Estadístico    [[55.400300427777374]]
P valor         4.268908724630835e-22
dtype: object

In [13]:
data["prediccion_educ"]= resultados_pe.predict()
data.head()

,inlf,hours,kidslt6,kidsge6,age,educ,wage,repwage,hushrs,husage,...,mtr,motheduc,fatheduc,unem,city,exper,nwifeinc,lwage,expersq,prediccion_educ
0,1,1610,1,0,32,12,3.3540,2.65,2708,34,...,0.7215,12,7,5.0,0,14,10.910060,1.210154,196,12.756017
1,1,1656,0,2,30,12,1.3889,2.65,2310,30,...,0.6615,7,7,11.0,1,5,19.499981,0.328512,25,11.733558
2,1,1980,1,3,35,12,4.5455,4.04,3072,40,...,0.6915,12,7,5.0,0,15,12.039910,1.514138,225,12.771979
3,1,456,0,3,34,12,1.0965,3.25,1920,53,...,0.7815,7,7,5.0,0,6,6.799996,0.092123,36,11.767683
4,1,1568,1,2,31,14,4.5918,3.60,2000,32,...,0.6215,12,14,9.5,1,7,20.100060,1.524272,49,13.914615


**Segunda Etapa**

In [14]:
Y1SE = data["lwage"] # Variable dependiente 
Y2SE = data["prediccion_educ"] 
XSE = data[["exper","expersq"]]

In [15]:
frame_SE = [XSE,Y2SE]
regresoras_SE = pd.concat(frame_SE, axis = 1)
regresoras_SE.head()

,exper,expersq,prediccion_educ
0,14,196,12.756017
1,5,25,11.733558
2,15,225,12.771979
3,6,36,11.767683
4,7,49,13.914615


In [16]:
segunda_etapa = sm.OLS(Y1SE,sm.add_constant(regresoras_SE))
resultados_SE = segunda_etapa.fit()
print(resultados_SE.summary())

                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.050
Model:                            OLS   Adj. R-squared:                  0.043
Method:                 Least Squares   F-statistic:                     7.405
Date:                Fri, 24 Jun 2022   Prob (F-statistic):           7.62e-05
Time:                        10:08:41   Log-Likelihood:                -457.17
No. Observations:                 428   AIC:                             922.3
Df Residuals:                     424   BIC:                             938.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.0481      0.420     

**Proceso Automático** 

In [17]:
from linearmodels.iv import IV2SLS

Y1 = data["lwage"] # Variable dependiente 
Y2 = data["educ"] # Variable endógena 
X = data[["exper","expersq"]] # Variables exógenas
Z = data[["motheduc","fatheduc"]] # Instrumentos

In [18]:
modelo_auto = IV2SLS(dependent= Y1, exog = sm.add_constant(X), endog= Y2, instruments= Z)
resultados_auto = modelo_auto.fit()
print(resultados_auto.summary)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                  lwage   R-squared:                      0.1357
Estimator:                    IV-2SLS   Adj. R-squared:                 0.1296
No. Observations:                 428   F-statistic:                    18.611
Date:                Fri, Jun 24 2022   P-value (F-stat)                0.0003
Time:                        10:08:42   Distribution:                  chi2(3)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          0.0481     0.4278     0.1124     0.9105     -0.7903      0.8865
exper          0.0442     0.0155     2.8546     0.00

**Con errores robustos**

In [19]:
frames_MCO = [X , Y2]
regresoras_MCO = pd.concat(frames_MCO, axis = 1)
regresoras_MCO

,exper,expersq,educ
0,14,196,12
1,5,25,12
2,15,225,12
3,6,36,12
4,7,49,14
...,...,...,...
423,2,4,10
424,21,441,12
425,22,484,13
426,14,196,12


In [20]:
modelo_MCO = sm.OLS(Y1, sm.add_constant(regresoras_MCO))
resultados_MCO = modelo_MCO.fit(cov_type = "HC0") 
print(resultados_MCO.summary())

                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.157
Model:                            OLS   Adj. R-squared:                  0.151
Method:                 Least Squares   F-statistic:                     27.56
Date:                Fri, 24 Jun 2022   Prob (F-statistic):           2.68e-16
Time:                        10:08:42   Log-Likelihood:                -431.60
No. Observations:                 428   AIC:                             871.2
Df Residuals:                     424   BIC:                             887.4
Df Model:                           3                                         
Covariance Type:                  HC0                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.5220      0.201     -2.601      0.0

**Comparar Modelo**

In [21]:
from linearmodels.iv import compare

In [22]:
# compare({"MCO": resultados_MCO , "2 Etapas" : regresoras_SE}) # Falla porque son de distinto tipo

In [23]:
resultado_MCO = IV2SLS(dependent= Y1, exog= sm.add_constant(regresoras_MCO), endog = None, instruments = None).fit()
print(resultado_MCO.summary)
# Calculamos MCO con misma función para poder hacer el compare 

                            OLS Estimation Summary                            
Dep. Variable:                  lwage   R-squared:                      0.1568
Estimator:                        OLS   Adj. R-squared:                 0.1509
No. Observations:                 428   F-statistic:                    82.671
Date:                Fri, Jun 24 2022   P-value (F-stat)                0.0000
Time:                        10:08:42   Distribution:                  chi2(3)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const         -0.5220     0.2007    -2.6010     0.0093     -0.9154     -0.1287
exper          0.0416     0.0152     2.7344     0.00

In [24]:
resultado_auto = IV2SLS(dependent= Y1, exog= sm.add_constant(X), endog= Y2, instruments= Z).fit()
print(resultado_auto.summary)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                  lwage   R-squared:                      0.1357
Estimator:                    IV-2SLS   Adj. R-squared:                 0.1296
No. Observations:                 428   F-statistic:                    18.611
Date:                Fri, Jun 24 2022   P-value (F-stat)                0.0003
Time:                        10:08:42   Distribution:                  chi2(3)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          0.0481     0.4278     0.1124     0.9105     -0.7903      0.8865
exper          0.0442     0.0155     2.8546     0.00

In [25]:
resultado_manual= IV2SLS(dependent=Y1, exog=sm.add_constant(regresoras_SE),endog=None, instruments=None).fit()
print(resultado_manual)

                            OLS Estimation Summary                            
Dep. Variable:                  lwage   R-squared:                      0.0498
Estimator:                        OLS   Adj. R-squared:                 0.0431
No. Observations:                 428   F-statistic:                    17.111
Date:                Fri, Jun 24 2022   P-value (F-stat)                0.0007
Time:                        10:08:42   Distribution:                  chi2(3)
Cov. Estimator:                robust                                         
                                                                              
                                Parameter Estimates                                
                 Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
-----------------------------------------------------------------------------------
const               0.0481     0.4492     0.1071     0.9147     -0.8323      0.9285
exper               0.0442     0

In [26]:
print(compare({"MCO": resultado_MCO, "2 Etapas Manual": resultado_manual, "2 Etapas Automático":resultado_auto}))

                           Model Comparison                           
                               MCO 2 Etapas Manual 2 Etapas Automático
----------------------------------------------------------------------
Dep. Variable                lwage           lwage               lwage
Estimator                      OLS             OLS             IV-2SLS
No. Observations               428             428                 428
Cov. Est.                   robust          robust              robust
R-squared                   0.1568          0.0498              0.1357
Adj. R-squared              0.1509          0.0431              0.1296
F-statistic                 82.671          17.111              18.611
P-value (F-stat)            0.0000          0.0007              0.0003
==================     ===========     ===========         ===========
const                      -0.5220          0.0481              0.0481
                         (-2.6010)        (0.1071)            (0.1124)
exper 